# What is the most optimal skill to learn as a Data Analyst

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ast

df = pd.read_csv("C:/Users/DELL/Downloads/data_jobs.csv")
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

In [2]:
df_us = df[df['job_country']=='United States'].dropna(subset='salary_year_avg').copy()

In [5]:
df_us = df_us.dropna(subset=['salary_year_avg'])

In [6]:
df_us_exploded = df_us.explode('job_skills')